In [9]:
import requests, uuid, json
import pyewts

## Translate English Sentences

In [11]:
with open('/home/j/bing-key.txt', 'r') as f:
    key = f.read()
key=key.replace('\n', '')

In [41]:
batch_num = 8

In [46]:
def translate(key, batch_num):
    # Add your key and endpoint

    endpoint = "https://api.cognitive.microsofttranslator.com"


    # location, also known as region.
    # required if you're using a multi-service or regional (not global) resource. It can be found in the Azure portal on the Keys and Endpoint page.
    location = "eastus"

    path = '/translate'
    constructed_url = endpoint + path

    params = {
        'api-version': '3.0',
        'from': 'en',
        'to': 'bo',
    }

    headers = {
        'Ocp-Apim-Subscription-Key': key,
        # location required if you're using a multi-service or regional (not global) resource.
        'Ocp-Apim-Subscription-Region': location,
        'Content-type': 'application/json',
        'X-ClientTraceId': str(uuid.uuid4())
    }

    # You can pass more than one object in body.
    body = []

    with open('/home/j/Documents/Projects/Iron-Bridge/lotsawa/data/ankhi/bing-batches/batch'+str(batch_num)+'.txt', 'r') as f:
        for line in f:
            body.append({'text': line})

    request = requests.post(constructed_url, params=params, headers=headers, json=body)
    response = request.json()
    #print(json.dumps(response, sort_keys=True, ensure_ascii=False, indent=4, separators=(',', ': ')))
    with open('/home/j/Documents/Projects/Iron-Bridge/lotsawa/data/ankhi/bing-translations/batch'+str(batch_num)+'-response.json', 'w', encoding='utf-8') as f:
        json.dump(response, f, ensure_ascii=False, indent=4)

## Transliterate Tibetan to Latin Script

In [47]:
def transliterate(batch_num):
    converter=pyewts.pyewts()

    with open('/home/j/Documents/Projects/Iron-Bridge/lotsawa/data/ankhi/bing-translations/batch'+str(batch_num)+'-response.json') as f:
        d = json.load(f)

    tib = []

    for entry in d:
        tib.append(converter.toWylie(entry['translations'][0]['text']))

    eng = []

    with open('/home/j/Documents/Projects/Iron-Bridge/lotsawa/data/ankhi/bing-batches/batch'+str(batch_num)+'.txt', 'r') as f:
        for line in f:
            eng.append(line)

    pairs = []

    for i in range(len(tib)):
        pairs.append((tib[i], eng[i]))

    with open('/home/j/Documents/Projects/Iron-Bridge/lotsawa/data/ankhi/pairs/batch'+str(batch_num)+'-pairs.txt', 'w') as f: # make sure to change to 'a'
        f.writelines('\n'.join(str(pair).replace('\'', '')
                                        .replace('\\n', '')
                                        .replace('(', '')
                                        .replace(')', '')
                                        .replace('/', '')
                                        .replace(' ,', ',')
                                        .replace('"', '')  for pair in pairs))

In [49]:
for i in range(45,120):
    batch_num = i
    translate(key, batch_num)
    transliterate(batch_num)

TypeError: string indices must be integers

## Concatenate Pairs

In [44]:
""" import os

for file in os.listdir('/home/j/Documents/Projects/Iron-Bridge/lotsawa/data/ankhi/pairs/'):
    path = '/home/j/Documents/Projects/Iron-Bridge/lotsawa/data/ankhi/pairs/' + file
    with open(path, 'r') as f:
        text = f.readlines()
        with open('/home/j/Documents/Projects/Iron-Bridge/lotsawa/data/ankhi/pairs/batch0-pairs.txt', 'a') as g:
            g.write('\n')
            g.writelines(text) """

IsADirectoryError: [Errno 21] Is a directory: '/home/j/Documents/Projects/Iron-Bridge/lotsawa/data/ankhi/pairs/minibatch-pairs'